## **1. Import Python API**
### **각 API 별 간단 설명**
* `torch.nn` : 모델링에 필요한 layer가 클래스로 정의되어 있어, 각 클래스를 이용해 모델 정의에 사용
* `torch.nn.functional` : 함수로 정의되어 있어, 인스턴스로 만들 필요 없이 함수를 선언하여 연산에 활용
* `torch.optim` : 다양한 optimization algorithms이 있는 package로 optimizer object를 선언하고, 연산된 gradient에 따라 parameter가 업데이트됨. 
* `torchvision` : cv에 사용되는 유명한 데이터셋, 모델 구조, 보편적인 이미지 tranformations 함수가 제공됨.
### **구현 내용**
1. api 선언 
2. cuda 텐서 사용 
* `torch.cuda`를 통해 CUDA 텐서 사용  
    * cpu 텐서와 같은 기능을 수행하지만 gpu 성능을 낼 수 있는 CUDA tensor type을 사용할 수 있음. 
    * `is_available()`함수를 통해 CUDA를 사용할지 말지 결정 
* cpu 및 gpu 연산 무작위 고정
    * 실험과정에서 시드가 무작위로 변한다면 모델 결과 확인에 어려움이 생김
    * 실험의 재현 가능성을 확보하기 위해 시드 고정

In [ ]:
# 1. API 선언
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

# 2. CUDA 텐서 사용 
# CUDA 텐서 사용 
device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.manual_seed(777) # for cpu
if device == 'cuda':
    torch.cuda.manual_seed_all(777) # for gpu 
print(device + " is available")

cuda is available


## **2. Settings**
### **구현 내용**
1. 모델 학습에 사용될 하이퍼파라미터 선언
* learning_rate(학습률): 경사하강법에서 가중치가 얼마나 갱신되는지 의미 
* batch_size(배치 사이즈): 전체 데이터를 여러 작은 그룹으로 나누었을때 (mini batch) 하나의 소그룹에 속하는 데이터의 수
    * 나누는 이유: 전체 데이터를 통째로 신경망에 넣으면 리소스 사용, 학습 등에 비효율 
    * 너무 크면 overfitting, 너무 작으면 local minima
* num_classes
* epochs: 전체 데이터가 신경망을 통과한 횟수 


In [ ]:
# 1. 하이퍼파라미터 선언
learning_rate = 0.001
batch_size = 100
num_classes = 10
epochs = 5

## **3. Load Data**
### **패키지 소개**
* `torchvision`
    * 유명한 데이터셋, 모델 아키텍쳐, common 이미지 transformer로 구성


### **구현 내용**
1. TrainSet과 TestSet을 구성 
* `torchvision.datasets.MNIST` 사용
    * MNIST 데이터셋 불러옴
    * `root`: (string) 불러온 데이터셋이 저장되는 directory 
    * `train`: (bool) True: Train 데이터, False: Test 데이터 (default: True)
    * `download`: (bool) True: 데이터 다운로드 후 root directory에 저장함 (default: False)
    * `transform`: (callable) function or transform, PIL 이미지를 transformed 된 version으로 바꿔서 반환 (default: None)
* `transfroms.Compose([transfroms.ToTensor()])`
    * PIL 이미지 혹은 Numpy `ndarray`를 `FloatTensor로 변환
    * 이미지의 pixel intensity(특정 화소의 밝은 정도)를 기존 [0,255]에서 [0.,1.,]범위로 스케일링 

2. train_loader, test_loader 생성 
* `torch.utils.data.DataLoader`
    * 역할
        * `Dataset`은 샘플과 정답(label)을 저장
        * `DataLoader`은 `Dataset`을 샘플에 쉽게 접근할 수 있도록 순회 가능한 객체로 감쌈
        * mini-batch, shuffle 등 옵션에 따라 학습 환경 설정 

In [ ]:
# 1. train_set, test_set 구성 
train_set = torchvision.datasets.MNIST(
    root = './data/MNIST',
    train = True,
    download = True,
    transform = transforms.Compose([
        transforms.ToTensor() 
    ])
)
test_set = torchvision.datasets.MNIST(
    root = './data/MNIST',
    train = False,
    download = True,
    transform = transforms.Compose([
        transforms.ToTensor()
    ])
)

# 2. train_loader, test_loader 생성 
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./data/MNIST/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./data/MNIST/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./data/MNIST/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./data/MNIST/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/MNIST/raw



## **4. Chcek Trainset**
### **구현 내용**
1. train_set의 dataset 확인
* `enumerate` 함수로 train_set을 iterable한 객체로(examples) 만듦
2. 객체의 첫번째 데이터 반환 
* 반환값은 (인덱스, tensor)의 튜플 형태로 각각 `batch_idx`, `(example_data, example_targets)`로 지정 
3. 각 데이터의 shape, 값 확인 

In [ ]:
# 1. train_set의 dataset 확인 
examples = enumerate(train_set)
# 2. 객체의 첫번째 데이터 반환 
batch_idx, (example_data, example_targets) = next(examples)

# 3. 각 데이터의 shape, 값 확인 
print('example_data.shape: {}'.format(example_data.shape)) # tensor shape
print('example_targets: {}'.format(example_targets)) # 분류 class
print('batch_idx: {}'.format(batch_idx)) # idx 확인: 첫번째라서 0 

example_data.shape: torch.Size([1, 28, 28])
example_targets: 5
batch_idx: 0


## **5. ConvNet class 선언**
### **구성**
* Input: 28*28 image 
* 2개의 Convolution Layer와 2개의 FC layer, 1개의 MaxPooling 레이어로 구성 


### **모듈 함수 상세**
#### **__init__()**
* `Conv2d`
    * in_channel: (int) input image의 채널 크기 
    * out_channel: (int) convolution 연산을 통해서 생성되는 채널의 크기
    * kernel_size: (int or tuple) convolution 커널의 크기
    * stride: (int or tuple) convolution 이동 폭
    * padding: (int, tuple or str) 각 변에 추가되는 padding 
* `drop2D`
    * p: (float) 모델 학습 시, 주어진 데이터 중 dropout시킬 데이터의 비율
    * inplcae: (bool) True라면 in-place 연산으로 수행, default: False
* `MaxPool2D`
    * kernel_size: (int) max over할 window size
* `Linear`
    * in_features: input sample의 크기
    * out_features: ouput sample의 크기


#### **forward**
* input x를 ConvNet에 넣어서 예측된 y 값을 반환하는 함수 
* 초기화때 선언해둔 각 레이어 함수를 활용해서 forward 함수 구성 

In [ ]:
class ConvNet(nn.Module):
  def __init__(self): 
        super(ConvNet, self).__init__() #부모 클래스 상속 받음. 

        self.conv1 = nn.Conv2d(1, 10, kernel_size=5) # in_channels: 1, out_channels: 10, kernel_size: 5, stride: 1, padding: 0, dilation: 1, groups: 1
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5) # in_channels: 10, out_channels: 20, kernel_size: 5, stride: 1, padding: 0, dilation: 1, groups: 1
        self.drop2D = nn.Dropout2d(p=0.25, inplace=False) # p: 0.25, inplace: False
        self.mp = nn.MaxPool2d(2)  # kernel_size: 1, padding: 0, dilation: 1
        self.fc1 = nn.Linear(320,100) # in_features: 320, out_features: 100
        self.fc2 = nn.Linear(100,10) # in_features: 100, out_features: 10 

  def forward(self, x): 
        x = F.relu(self.mp(self.conv1(x))) # conv1 layer에 relu 연산 (활성함수)
        x = F.relu(self.mp(self.conv2(x))) # conv2 layer에 relu 연산 (활성함수)
        x = self.drop2D(x) # drop out 
        x = x.view(x.size(0), -1) # flatten 
        x = self.fc1(x) # fc1 레이어 연산
        x = self.fc2(x) # fc2 레이어 연산 
        return F.log_softmax(x) 

## **6. Model Train**

### **1. Train 객체 선언**
* model: ConvNet 클래스로 model 선언
* criterion: Cost Function 선언
    * `nn.CrossEntropyLoss`: input과 target 간의 cross entropy loss를 계산 
* optimizer: 최적화 함수 선언
    * `torch.optim.Adam` : Adam 최적화 알고리즘을 적용한 optimizer 선언 


### **2. Train 시작** 
* 선언한 epochs(5)만큼 epoch 반복, 각 epoch마다


(1) train_loader에서 각 data, target 가져옴


(2) 예측값, cost 연산 


(3) 모델 파라미터 갱신 


(4) 평균 cost 계산


### **깨알** 
* `my_tensor.to(device)`와 같이 호출하면 기존에 설정한 `device`에 `my_tensor`의 복사본이 반환되어 해당 `device`에서 연산을 수행함. 이때, 덮어쓰는 연산은 아니므로 `my_tensor = ...`와 같이 직접 덮어써줘야함. 

In [ ]:
# 1. Train 객체 선언 
model = ConvNet().to(device) # model 선언 
criterion = nn.CrossEntropyLoss().to(device) # Cost function
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate) # optimizer 

# 2. Train 시작 
for epoch in range(epochs): 
    avg_cost = 0 # 매 epoch 마다 반복(초기화)

    for data, target in train_loader: # 각 train_loader내 데이터에 대해서 
        # (1) train_loader에서 각 data, target 가져옴
        data = data.to(device)
        target = target.to(device)
        
        # (2) 예측값, cost 연산 
        hypothesis = model(data) # 예측 y 값 반환 (hypothesis)
        cost = criterion(hypothesis, target) # cost_function을 통해 실제값과 예측값 사이의 cost 계산

        # (3) 모델 파라미터 갱신 
        optimizer.zero_grad() # 미분값이 누적되지 않도록 gradient를 모두 0으로 초기화
        cost.backward() # backward를 통해 gradient 계산
        optimizer.step()  # 모델의 파라미터 갱신 

        # (4) 평균 cost 계산 
        avg_cost += cost / len(train_loader) # 평균 cost 계산 

    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


[Epoch:    1] cost = 0.312649876
[Epoch:    2] cost = 0.112475462
[Epoch:    3] cost = 0.0857186466
[Epoch:    4] cost = 0.0762623101
[Epoch:    5] cost = 0.0664585084


## **7. Evaluate Model**
### **1. evaluation으로 전환**
* `model.eval()` : 특정 모듈에 대해서만 효과가 있는 것으로, 학습시 사용되는 Dropout, BatchNorm을 비활성화


### **2. Auto_grad 모드 차단**
* `torch.no_grad()`: 모든 연산에서 gradient 연산을 차단하여 메모리 소모를 줄여줌. 


### **3. 각 test_loader의 데이터에 대해서**
(1) 데이터 및 실제값 로드

(2) 학습모델을 통해서 out 예측

(3) 전체 및 맞춘 개수 저장 

### **4. Test Accuracy**
* 정확도(%) 계산: 100 * (맞춘 개수)/(전체 개수)

In [ ]:
# 1. evaluation으로 전환
model.eval()

with torch.no_grad(): # 2. Auto_grad 모드 차단 
    correct = 0 # 맞은 문제 
    total = 0 # 전체 문제 

    for data, target in test_loader: # 3. test_loader의 각 데이터에 대해서 
        # (1) 데이터 및 실제값 로드 
        data = data.to(device) 
        target = target.to(device) 

        # (2) 학습 모델을 통해서 out 예측 
        out = model(data) # model을 통해서 반환된 값
        preds = torch.max(out.data, 1)[1] # out 중 가장 큰 값이 예측값

        # (3) 전체 및 맞춘 개수 저장 
        total += len(target)  # 전체 클래스 개수 
        correct += (preds==target).sum().item() # 예측값과 실제값이 같은 경우 sum 
        
    print('Test Accuracy: ', 100.*correct/total, '%') # 정확도 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Test Accuracy:  98.53 %
